This notebook builds a function such that:
- input: a genre occurring in our data set
- output: list of all artists with that genre

Run all the cells leading up to the function and then you can put a genre into the function and run that cell.


This function will be turned into a web app using streamlit for publisc exploration of the dataset

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline
#%matplotlib notebook

import re

from functools import partial

Import the cleaned data:

In [2]:
%ls -lt ../../data/genre_lists/data_ready_for_model/

total 39824
-rw-r--r--  1 Daniel  staff    21724 Jun  9 11:47 genre_label_counts_TOTAL_2020-05-18-10-06.csv
-rw-r--r--@ 1 Daniel  staff   287510 Jun  4 13:42 genre_stats.html
-rw-r--r--@ 1 Daniel  staff   911587 Jun  4 13:39 genre_set_counts.html
-rw-r--r--@ 1 Daniel  staff     1845 Jun  4 13:11 female_bias_freq500.html
-rw-r--r--@ 1 Daniel  staff     1459 Jun  4 13:11 male_bias_freq500.html
-rw-r--r--  1 Daniel  staff    73746 May 29 10:19 genre_stats.csv
-rw-r--r--  1 Daniel  staff    66235 May 21 11:00 promiscuity_table.csv
-rw-r--r--  1 Daniel  staff    57474 May 20 12:47 corpus.mm.index
-rw-r--r--  1 Daniel  staff   382436 May 20 12:47 corpus.mm
-rw-r--r--  1 Daniel  staff    49966 May 20 12:47 genre_dictionary.dict
drwxr-xr-x  5 Daniel  staff      160 May 20 10:59 logistic_model_data/
-rw-r--r--  1 Daniel  staff    10926 May 18 11:10 genre_label_non-lonely_TRAINING_2020-05-18-10-06.csv
-rw-r--r--  1 Daniel  staff     8664 May 18 11:09 genre_label_lonely_TRAINING_2020-05-18-10-06.

In [3]:
%store -r now
now
#now = '2020-05-11-14-35'

'2020-05-18-10-06'

In [4]:
X_train = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_X_train_{}.csv'.format(now), index_col = ['artist'])
y_train = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_y_train_{}.csv'.format(now), index_col = ['artist'])
X_test = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_X_test_{}.csv'.format(now), index_col = ['artist'])
y_test = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_y_test_{}.csv'.format(now), index_col = ['artist'])

In [5]:
X_tot = pd.concat([X_train,X_test])
y_tot = pd.concat([y_train,y_test])

In [6]:
X_tot.shape, y_tot.shape

((15470, 2), (15470, 1))

In [7]:
data = y_tot.join([X_tot], how = 'outer')

In [8]:
data.head()

,gender,genrelist,genrelist_length
artist,,,
Pablo_Holman,male,"['pop', 'rock', 'emo_pop']",3
Bobby_Edwards,male,['country'],1
La_Palabra,male,"['afro_cuban_jazz', 'son_montuno', 'guaracha',...",4
Sherrick,male,"['r_and_b', 'soul']",2
Allen_Collins,male,['southern_rock'],1


In [9]:
data.shape, data.isnull().sum()

((15470, 3),
 gender              0
 genrelist           0
 genrelist_length    0
 dtype: int64)

### Genre Labels

Each value of the genre column is a _string_ of comma separated genre labels. We want to convert it to a _list_ of strings.

In [10]:
"""This function takes in a string of the form
appearing in the genrelist of the dataframe.
It strips the square brackets and extra quotes and
returns a list of strings where each string is a genre label."""
def genrelist(string):
    string = string.strip("[").strip("]").replace("'","")
    L = [s for s in string.split(',')]
    L_new = []
    for x in L:
        L_new.append(x.replace(" ","_").lstrip("_").rstrip("_"))
    while (str("") in L_new):
        L_new.remove("")
    return L_new

Now we apply it to the whole column and put the lists in a new column:

In [11]:
data['genrelist']= data['genrelist'].apply(genrelist)

In [12]:
data.head()

,gender,genrelist,genrelist_length
artist,,,
Pablo_Holman,male,"[pop, rock, emo_pop]",3
Bobby_Edwards,male,[country],1
La_Palabra,male,"[afro_cuban_jazz, son_montuno, guaracha, salsa...",4
Sherrick,male,"[r_and_b, soul]",2
Allen_Collins,male,[southern_rock],1


### Import the genre labels from the whole data set:

In [13]:
genrelist_df = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/genre_list_{}.csv'.format(now), index_col = 'Unnamed: 0')

In [14]:
genrelist_df.shape

(1494, 1)

## Co-Occurrence

Function to apply to dataframe that marks a row as having a genre label:

current version:

In [15]:
def artists_with_label(row, label = 'soul'):
    if label in row.genrelist:
        return True
    else:
        return False

def genre_artists(data, label = 'soul'):
    artists_with = partial(artists_with_label,label = label) # create the partial function for the selected genre
    data[label] = data.apply(artists_with, axis = 1) # select those artists with the selected genre
    return data[data[label]].index.sort_values() # produce alphabetical list of artists with the selected genre

prototype:

In [76]:
def artists_with_label(row, label = 'soul'):
    if label in row.genrelist:
        return True
    else:
        return False

def genre_artists(data, label = 'soul'):
    artists_with = partial(artists_with_label,label = label) # create the partial function for the selected genre
    data[label] = data.apply(artists_with, axis = 1) # select those artists with the selected genre
    artists = data[data[label]].reset_index()
    artists = pd.DataFrame(artists.artist.sort_values())
    artists.reset_index(inplace = True, drop = True)
    artists.columns = ["Artists labeled with {}".format(label)]
    return artists # produce alphabetical list of artists with the selected genre

In [77]:
artists = genre_artists(data, 'punk')

In [78]:
artists

,Artists labeled with punk
0,Alexander_Rocciasana
1,Alfunction
2,Barry_Donegan
3,Baz_Warne
4,Becky_Stark
5,Billy_Karren
6,Cass_McCombs
7,Chris_Bailey
8,Chris_Clavin
9,Chris_Eskola


In [74]:
mylist = ['a','c','b','aa']
ml = sorted(mylist)
ml

['a', 'aa', 'b', 'c']